In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

In [2]:
!pip install flair

     |████████████████████████████████| 143kB 2.7MB/s 
  Stored in directory: /tmp/.cache/pip/wheels/bd/57/d3/907c3ee02d35e66f674ad0106e61f06eeeb98f6ee66a6cc3fe
  Stored in directory: /tmp/.cache/pip/wheels/15/ee/a8/6112173f1386d33eebedb3f73429cfa41a4c3084556bcee254
Successfully built sqlitedict segtok


In [3]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [4]:
import re
RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
def clean_tweet(tweet):
    tweet = re.sub('http\S+\s*', '', tweet)  # remove URLs
    tweet = re.sub('RT|cc', '', tweet)  # remove RT and cc
    tweet = re.sub('#', '', tweet)  # remove hashtags
    tweet = re.sub('@\S+', '', tweet)  # remove mentions
    tweet = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), '', tweet)  # remove punctuations
    tweet = re.sub('\s+', ' ', tweet)  # remove extra whitespace
    tweet = re.sub('\d+',' ',tweet) #removes numbers
    return tweet
  

train['clean_tweet'] = train['tweet'].apply(lambda x: clean_tweet(x))
test['clean_tweet'] = test['tweet'].apply(lambda x: clean_tweet(x))
train.head()

,id,label,tweet,clean_tweet
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so self...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit i cant use cause they ...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in urð...
4,5,0,factsguide: society now #motivation,factsguide society now motivation


In [5]:
import torch
from flair.data import Sentence
from flair.embeddings import ELMoEmbeddings
from flair.embeddings import FlairEmbeddings, BertEmbeddings
flair_forward_embedding = FlairEmbeddings('multi-forward')
flair_backward_embedding = FlairEmbeddings('multi-backward')
bert_embedding = BertEmbeddings('bert-base-cased')
elmo_embedding = ELMoEmbeddings('original')

from flair.embeddings import StackedEmbeddings
stacked_embeddings = StackedEmbeddings(
    embeddings=[flair_forward_embedding, flair_backward_embedding, bert_embedding,elmo_embedding])

2019-06-29 04:51:28,315 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings-v0.4/lm-multi-forward-v0.1.pt not found in cache, downloading to /tmp/tmpih3ee8cp


100%|██████████| 73034300/73034300 [00:09<00:00, 8067015.58B/s]

2019-06-29 04:51:38,540 copying /tmp/tmpih3ee8cp to cache at /tmp/.flair/embeddings/lm-multi-forward-v0.1.pt


2019-06-29 04:51:38,621 removing temp file /tmp/tmpih3ee8cp
2019-06-29 04:51:46,207 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings-v0.4/lm-multi-backward-v0.1.pt not found in cache, downloading to /tmp/tmpm4c4x4da


100%|██████████| 73034304/73034304 [00:08<00:00, 8156897.41B/s]

2019-06-29 04:51:56,320 copying /tmp/tmpm4c4x4da to cache at /tmp/.flair/embeddings/lm-multi-backward-v0.1.pt
2019-06-29 04:51:56,423 removing temp file /tmp/tmpm4c4x4da


2019-06-29 04:51:56,725 The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


100%|██████████| 404400730/404400730 [00:32<00:00, 12604790.00B/s]
/opt/conda/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/allennlp/commands/find_learning_rate.py:55: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  import matplotlib; matplotlib.use('Agg')
100%|██████████| 374434792/374434792 [00:24<00:00, 15496259.43B/s]


In [6]:
sentence = Sentence('NLP is lit!')
# embed words in sentence #
stacked_embeddings.embed(sentence)
for token in sentence:
    print(token.embedding)
# data type and size of embedding #
    print(type(token.embedding))
# storing size (length) #
z = token.embedding.size()[0]

tensor([ 1.6421e-03,  4.3796e-07, -8.6378e-07,  ...,  1.9982e-01,
        -3.7610e-01,  5.4207e-01])
<class 'torch.Tensor'>
tensor([-4.1463e-04,  3.7489e-07,  1.5540e-05,  ..., -3.0025e-01,
         1.3170e+00,  1.0456e+00])
<class 'torch.Tensor'>
tensor([-9.5101e-04,  3.4755e-08, -2.8645e-06,  ..., -8.2120e-02,
         4.0707e-01,  8.1440e-01])
<class 'torch.Tensor'>


In [7]:
trainvec = []
for i in range(len(train)):
    sent = torch.zeros(0,z)
    sentence = Sentence(train['clean_tweet'][i])
    stacked_embeddings.embed(sentence)
    word = torch.zeros(0,z)
    for token in sentence:
        word = torch.cat((word,token.embedding.view(-1,z)),0)
    sent = torch.cat((sent, word.mean(dim = 0).view(-1, z)),0)
    trainvec.append(sent)
train['embedding'] = trainvec
        

In [8]:
import pickle
stack_train_new = np.concatenate(train['embedding'], axis = 0)
pickle_out = open("stack_train_03032019.pickle","wb")
pickle.dump(stack_train_new, pickle_out)
pickle_out.close()


In [9]:
testvec = []
append = testvec.append
for i in range(len(test)):
    sent = torch.zeros(0,z)
    sentence = Sentence(test['clean_tweet'][i])
    stacked_embeddings.embed(sentence)
    word = torch.zeros(0,z)
    for token in sentence:
        word = torch.cat((word,token.embedding.view(-1,z)),0)
    sent = torch.cat((sent, word.mean(dim = 0).view(-1, z)),0)
    append(sent)
test['embedding'] = testvec

In [10]:
stack_test_new = np.concatenate(test['embedding'], axis = 0)
pickle_out = open("stack_test_03032019.pickle","wb")
pickle.dump(stack_test_new, pickle_out)
pickle_out.close()
test.head()

,id,tweet,clean_tweet,embedding
0,31963,#studiolife #aislife #requires #passion #dedic...,studiolife aislife requires passion dedication...,"[[tensor(-0.0030), tensor(5.1520e-07), tensor(..."
1,31964,@user #white #supremacists want everyone to s...,white supremacists want everyone to see the n...,"[[tensor(-0.0017), tensor(7.3739e-07), tensor(..."
2,31965,safe ways to heal your #acne!! #altwaystohe...,safe ways to heal your acne altwaystoheal heal...,"[[tensor(-0.0040), tensor(1.6659e-06), tensor(..."
3,31966,is the hp and the cursed child book up for res...,is the hp and the cursed child book up for res...,"[[tensor(-0.0028), tensor(6.2595e-07), tensor(..."
4,31967,"3rd #bihday to my amazing, hilarious #nephew...",rd bihday to my amazing hilarious nephew eli...,"[[tensor(-0.0005), tensor(7.1744e-07), tensor(..."


In [11]:
#load stack_train_new
pickle_in = open("stack_train_03032019.pickle", "rb")
stack_train_new = pickle.load(pickle_in)

#load stack_train_new
pickle_in = open("stack_test_03032019.pickle", "rb")
stack_test_new = pickle.load(pickle_in)

In [12]:
stack_train_new = np.where(np.isfinite(stack_train_new), stack_train_new, 0)

In [13]:
stack_test_new = np.where(np.isfinite(stack_test_new), stack_test_new, 0)

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
#from sklearn.metrics import accuracy_score
X = stack_train_new
y = train['label']
i=1 
kf = StratifiedKFold(n_splits=3,random_state=1,shuffle=True) 
for train_index,test_index in kf.split(X,y):     
    #print('\n{} of kfold {}'.format(i,kf.n_splits))    
    xtr,xvl = X[train_index],X[test_index]     
    ytr,yvl = y[train_index],y[test_index]         
    model1 = LogisticRegression(solver='lbfgs',max_iter=1500)     
    model1.fit(xtr, ytr)     
#     pred_test1 = model_svc.predict(xvl)    
#     score = accuracy_score(yvl,pred_test1)     
#     print('accuracy_score',score)    
    i+=1

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [15]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold
#from sklearn.metrics import accuracy_score
X = stack_train_new
y = train['label']
i=1 
kf = StratifiedKFold(n_splits=3,random_state=1,shuffle=True) 
for train_index,test_index in kf.split(X,y):     
    #print('\n{} of kfold {}'.format(i,kf.n_splits))    
    xtr,xvl = X[train_index],X[test_index]     
    ytr,yvl = y[train_index],y[test_index]         
    model_svc = LinearSVC(max_iter=500)     
    model_svc.fit(xtr, ytr)     
#     pred_test1 = model_svc.predict(xvl)    
#     score = accuracy_score(yvl,pred_test1)     
#     print('accuracy_score',score)    
    i+=1

/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [16]:
preds_test = model1.predict(stack_test_new)
sub = pd.DataFrame({'id':test['id'], 'label':preds_test})

# write predictions to a CSV file
sub.to_csv("lreg3.csv", index=False)

In [17]:
preds_test = model_svc.predict(stack_test_new)
sub = pd.DataFrame({'id':test['id'], 'label':preds_test})

# write predictions to a CSV file
sub.to_csv("svc.csv", index=False)